In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import geopandas as gpd
import requests

from zipfile import ZipFile

from shapely.geometry import Point, Polygon

In [2]:
def konwersja_wspolrzedne(stopnie, minuty, sekundy, kierunek):
    wspolrzedne = stopnie + (minuty / 60) + (sekundy / 3600)
    return wspolrzedne

#### TERYT 

In [3]:
TERYT = pd.read_csv('TERC_Urzedowy_2025-07-03/TERC_Urzedowy_2025-07-03.csv', sep=';', dtype={'WOJ': 'str', 'POW': 'str', 'GMI': 'str', 'RODZ': 'str'})

In [4]:
TERYT

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2025-01-01
1,02,01,NaN,NaN,bolesławiecki,powiat,2025-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2025-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2025-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2025-01-01
...,...,...,...,...,...,...,...
4343,32,61,01,1,Koszalin,gmina miejska,2025-01-01
4344,32,62,NaN,NaN,Szczecin,miasto na prawach powiatu,2025-01-01
4345,32,62,01,1,Szczecin,gmina miejska,2025-01-01
4346,32,63,NaN,NaN,Świnoujście,miasto na prawach powiatu,2025-01-01


In [ ]:
TERYT['ID_WOJ'] = TERYT['WOJ']
TERYT['ID_POW'] = TERYT['WOJ']+TERYT['POW']
TERYT['ID_GMI'] = TERYT['WOJ']+TERYT['POW']+TERYT['GMI']+TERYT['RODZ']

In [ ]:
WOJ = TERYT[TERYT['POW'].isnull()]
WOJ = WOJ.drop(columns={'ID_POW', 'ID_GMI','WOJ', 'POW', 'GMI', 'RODZ', 'STAN_NA'})
POW = TERYT[(TERYT['GMI'].isnull()) & (TERYT['POW'].notnull())]
POW = POW.drop(columns={'WOJ', 'POW', 'GMI', 'RODZ', 'STAN_NA', 'ID_GMI'})
GMI = TERYT[(TERYT['GMI'].notnull()) & (TERYT['RODZ'].notnull())]
GMI = GMI.drop(columns={'WOJ','POW', 'GMI', 'STAN_NA', 'ID_WOJ'})

In [ ]:
WOJ.to_csv('WOJ.csv', index=False, sep=',', encoding='Windows 1250')
POW.to_csv('POW.csv', index=False, sep=',', encoding='Windows 1250')
GMI.to_csv('GMI.csv', index=False, sep=',', encoding='Windows 1250')

#### DANE 

In [5]:
## ID, NAZWA ORAZ KOD STACJI
WYKAZ_STACJI = pd.read_csv('wykaz_stacji.csv', encoding='Windows 1250', header=None, names=['ID', 'NAZWA', 'KOD'], dtype={'KOD': 'object', 'ID': 'object'})

WYKAZ_STACJI['KOD'] = WYKAZ_STACJI['KOD'].str.strip()

In [7]:
WYKAZ_STACJI

,ID,NAZWA,KOD
0,250180460,ADAMOWICE,95414
1,254230010,ALEKSANDRÓWKA,91908
2,250190430,ALWERNIA,95506
3,250210030,ANNOPOL,95751
4,249199978,ANTAŁÓWKA,6522
...,...,...,...
2157,253190270,ŻUROMIN,92504
2158,252190120,ŻYCHLIN,93514
2159,254220010,ŻYTKIEJMY,91820
2160,249199975,ŻYWIEC,96597


In [8]:
# WSPOLRZEDNE GPS STACJI WRAZ KODEM I NAZWA
STACJE = pd.read_excel("STACJE.xlsx", header=None)
STACJE.columns = ['KOLUMNA']

In [9]:
STACJE

,KOLUMNA
0,3150 BABIMOST 15 47 52 08 OBRZYCA ---------- -...
1,3152 BABIMOST 15 47 46 52 08 24 OBRZYCA ------...
2,6808 BALIGRÓD-MCHAWA 22 17 49 21 JABŁONKA ----...
3,6708 BARWINEK 21 41 49 26 PANNA -----KKKKK KKK...
4,4505 BEŁCHATÓW 19 22 51 24 WIDAWKA ---------- ...
...,...
334,400 ZIELONA 15 31 28 51 55 49 ODRA KKKKKKKKKK ...
335,4206 ZYBISZÓW 16 55 51 04 ODRA ---------- ----...
336,6701 ŻARNOWA 21 49 49 53 WISŁOK ---------- KKK...
337,6851 ŻUBRACZE 22 16 07 49 12 23 SAN ----------...


In [10]:
# WYCIĄGNIECIE WSPOLRZEDNYCH - AKTUALNY FORMAT SIE NIE NADAWAL
STACJE["KOD"] = STACJE['KOLUMNA'].str.split(' ').str[0]
STACJE["NAZWA"] = STACJE['KOLUMNA'].str.split(' ').str[1]

STACJE["_1"] = STACJE['KOLUMNA'].str.split(' ').str[2]
STACJE["_2"] = STACJE['KOLUMNA'].str.split(' ').str[3]
STACJE["_3"] = STACJE['KOLUMNA'].str.split(' ').str[4]
STACJE["_4"] = STACJE['KOLUMNA'].str.split(' ').str[5]
STACJE["_5"] = STACJE['KOLUMNA'].str.split(' ').str[6]
STACJE["_6"] = STACJE['KOLUMNA'].str.split(' ').str[7]

STACJE['D1'] = STACJE['_1'].astype(int)
STACJE['D2'] = STACJE['_2'].astype(int)
STACJE['D3'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_3'], '0').astype(int)

STACJE['S1'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_4'], STACJE['_3']).astype(int)
STACJE['S2'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_5'], STACJE['_4']).astype(int)
STACJE['S3'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_6'], '0').astype(int)

# STACJE[~STACJE['D1'].apply(lambda x: str(x).isdigit())]

STACJE['LAT'] = konwersja_wspolrzedne(STACJE['S1'], STACJE['S2'], STACJE['S3'], 'N')
STACJE['LON'] = konwersja_wspolrzedne(STACJE['D1'], STACJE['D2'], STACJE['D3'], 'N')

In [19]:
STACJE

,KOLUMNA,KOD,NAZWA,_1,_2,_3,_4,_5,_6,D1,D2,D3,S1,S2,S3,LAT,LON
0,3150 BABIMOST 15 47 52 08 OBRZYCA ---------- -...,3150,BABIMOST,15,47,52,08,OBRZYCA,----------,15,47,0,52,8,0,52.133333,15.783333
1,3152 BABIMOST 15 47 46 52 08 24 OBRZYCA ------...,3152,BABIMOST,15,47,46,52,08,24,15,47,46,52,8,24,52.140000,15.796111
2,6808 BALIGRÓD-MCHAWA 22 17 49 21 JABŁONKA ----...,6808,BALIGRÓD-MCHAWA,22,17,49,21,JABŁONKA,-----KKKKK,22,17,0,49,21,0,49.350000,22.283333
3,6708 BARWINEK 21 41 49 26 PANNA -----KKKKK KKK...,6708,BARWINEK,21,41,49,26,PANNA,-----KKKKK,21,41,0,49,26,0,49.433333,21.683333
4,4505 BEŁCHATÓW 19 22 51 24 WIDAWKA ---------- ...,4505,BEŁCHATÓW,19,22,51,24,WIDAWKA,----------,19,22,0,51,24,0,51.400000,19.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,400 ZIELONA 15 31 28 51 55 49 ODRA KKKKKKKKKK ...,400,ZIELONA,15,31,28,51,55,49,15,31,28,51,55,49,51.930278,15.524444
335,4206 ZYBISZÓW 16 55 51 04 ODRA ---------- ----...,4206,ZYBISZÓW,16,55,51,04,ODRA,----------,16,55,0,51,4,0,51.066667,16.916667
336,6701 ŻARNOWA 21 49 49 53 WISŁOK ---------- KKK...,6701,ŻARNOWA,21,49,49,53,WISŁOK,----------,21,49,0,49,53,0,49.883333,21.816667
337,6851 ŻUBRACZE 22 16 07 49 12 23 SAN ----------...,6851,ŻUBRACZE,22,16,07,49,12,23,22,16,7,49,12,23,49.206389,22.268611


In [17]:
LISTA_STACJE = (WYKAZ_STACJI.merge(STACJE[['KOD', 'LAT', 'LON']], 'left', on='KOD', indicator=True)
                .loc[lambda x: x['_merge'] == 'both']
                .reset_index(drop=True)
                .drop(columns=['_merge'])
               )


In [18]:
LISTA_STACJE

,ID,NAZWA,KOD,LAT,LON
0,252159995,BABIMOST,3150,52.133333,15.783333
1,252150270,BABIMOST,3152,52.140000,15.796111
2,249220170,BALIGRÓD-MCHAWA,6808,49.350000,22.283333
3,249210240,BARWINEK,6708,49.433333,21.683333
4,251199995,BEŁCHATÓW,4505,51.400000,19.366667
...,...,...,...,...,...
334,351150400,ZIELONA GÓRA,400,51.930278,15.524444
335,251169995,ZYBISZÓW,4206,51.066667,16.916667
336,249219996,ŻARNOWA,6701,49.883333,21.816667
337,249229989,ŻUBRACZE,6851,49.206389,22.268611


#### SHAPEFILE

In [12]:
shp_w = gpd.read_file('00_jednostki_administracyjne/A01_Granice_wojewodztw.shp')
shp_p = gpd.read_file('00_jednostki_administracyjne/A02_Granice_powiatow.shp')
shp_g = gpd.read_file('00_jednostki_administracyjne/A03_Granice_gmin.shp')

In [13]:
for stacja in range(LISTA_STACJE.shape[0]):
    # print(stacja)
    point = Point(tuple(LISTA_STACJE.loc[stacja][['LON', 'LAT']]))
    for wojewodztwo in range(shp_w.shape[0]):
        wojewodztwo_teryt = shp_w.loc[wojewodztwo, 'JPT_KOD_JE']
        polygon = shp_w.loc[wojewodztwo, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_WOJ'] = wojewodztwo_teryt
    for powiat in range(shp_p.shape[0]):
        powiat_teryt = shp_p.loc[powiat, 'JPT_KOD_JE']
        polygon = shp_p.loc[powiat, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_POW'] = powiat_teryt
    for gmina in range(shp_g.shape[0]):
        gmina_teryt = shp_g.loc[gmina, 'JPT_KOD_JE']
        polygon = shp_g.loc[gmina, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_GMI'] = gmina_teryt

C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_312\1002392590.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'ID_WOJ'] = wojewodztwo_teryt
C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_312\1002392590.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0809' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'ID_POW'] = powiat_teryt
C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_312\1002392590.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0809013' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'ID_GMI'

In [14]:
## stacja w darłowie znajduje się na morzu poza zakresem shapefile
LISTA_STACJE = LISTA_STACJE.fillna({'ID_WOJ': '32', 'ID_POW': '3213', 'ID_GMI': '3213011'})

In [15]:
LISTA_STACJE

,ID,NAZWA,KOD,LAT,LON,ID_WOJ,ID_POW,ID_GMI
0,252159995,BABIMOST,3150,52.133333,15.783333,08,0809,0809013
1,252150270,BABIMOST,3152,52.140000,15.796111,08,0809,0809013
2,249220170,BALIGRÓD-MCHAWA,6808,49.350000,22.283333,18,1821,1821012
3,249210240,BARWINEK,6708,49.433333,21.683333,18,1807,1807023
4,251199995,BEŁCHATÓW,4505,51.400000,19.366667,10,1001,1001022
...,...,...,...,...,...,...,...,...
334,351150400,ZIELONA GÓRA,400,51.930278,15.524444,08,0862,0862011
335,251169995,ZYBISZÓW,4206,51.066667,16.916667,02,0223,0223043
336,249219996,ŻARNOWA,6701,49.883333,21.816667,18,1819,1819043
337,249229989,ŻUBRACZE,6851,49.206389,22.268611,18,1821,1821022


In [ ]:
(LISTA_STACJE
 .to_csv('LISTA_STACJE.csv', index=False, sep=',', encoding='Windows 1250')
)

#### DANE METEO 

In [ ]:
for rok in [str(x) for x in np.arange(2001, 2026, 1)]:
    for msc in ["01", "02", "03", "04" ,"05",'06',"07",'08','09','10','11','12']:
        LINK = f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/{rok}/{rok}_{msc}_k.zip"
        PLIK = requests.get(LINK).content

        with open(f"{rok+"_"+msc}.zip", "wb") as file:
            file.write(PLIK)
    print(rok)


In [ ]:
for plik in os.listdir('PLIKI_NOWE/'):
    # print(plik)
    with ZipFile(f'PLIKI_NOWE/{plik}', 'r') as plik_zip:
        try:
            plik_zip.extractall('C:\\PythonProjects\\meteo\\PLIKI_CSV')
        except:
            print(plik)
            print('blad')
            pass
    

In [ ]:
df = pd.DataFrame()
for plik in os.listdir('PLIKI_CSV/'):
    if plik[:5] != 'k_d_t':
        df_temp = pd.read_csv(f'PLIKI_CSV/{plik}', encoding='Windows 1250', header=None, usecols=[0,1,2,3,4,5,7,9, 13], names=['ID', 'NAZWA', 'ROK', 'MIESIAC', 'DZIEN', 'TEMP_MAX', 'TEMP_MIN', 'TEMP_MEAN', 'PREC'], dtype={'ID': 'object'})
        
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)

In [ ]:
df.drop(columns=['NAZWA']).to_csv('DANE_METEO.csv', index=False, sep=',', encoding='Windows 1250')

#### SQL 

In [ ]:
conn = sqlite3.connect("METEO_DB.sqlite")
cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE WOJ (
    ID_WOJ   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL);
    """)

cursor.execute("""CREATE TABLE POW (
    ID_POW   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL,
    ID_WOJ TEXT NOT NULL,
    FOREIGN KEY (ID_WOJ)
       REFERENCES WOJ (ID_WOJ) );
    """)

cursor.execute("""CREATE TABLE GMI (
    ID_GMI   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL,
    RODZ TEXT NOT NULL,
    ID_POW TEXT NOT NULL,
    FOREIGN KEY (ID_POW)
       REFERENCES POW (ID_POW) );
    """)

cursor.execute("""CREATE TABLE STACJE (
    ID_STACJA   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    KOD      TEXT NOT NULL,
    LAT REAL NOT NULL,
    LON REAL NOT NULL,
    ID_GMI TEXT NOT NULL,
    FOREIGN KEY (ID_GMI) REFERENCES GMI (ID_GMI)
    );
    """)

cursor.execute("""CREATE TABLE METEO (
    METEO_ID   INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_STACJA TEXT    NOT NULL,
    ROK      TEXT NOT NULL,
    MIESIAC TEXT NOT NULL,
    DZIEN TEXT NOT NULL,
    TEMP_MAX REAL NOT NULL,
    TEMP_MIN REAL NOT NULL,
    TEMP_MEAN REAL NOT NULL,
    PREC REAL NOT NULL,
    FOREIGN KEY (ID_STACJA) REFERENCES STACJE (ID_STACJA)
    );
    """)

In [ ]:
conn = sqlite3.connect("METEO_DB.sqlite")
cursor = conn.cursor()

In [ ]:
WOJ.to_sql('WOJ', conn, if_exists='append', index=False)
POW.to_sql('POW', conn, if_exists='append', index=False)
GMI.to_sql('GMI', conn, if_exists='append', index=False)

In [ ]:
LISTA_STACJE.rename(columns={'ID': 'ID_STACJA'}).to_sql('STACJE', conn, if_exists='append', index=False)

In [ ]:
df.fillna(-1000).rename(columns={'ID': "ID_STACJA"}).drop(columns=['NAZWA']).to_sql('METEO', conn, if_exists='append', index=False)